<a href="https://colab.research.google.com/github/Aasthapaudel/-Imagine-cup-cloud-skill-Challenge-/blob/main/RAG_c2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1
!pip install -q transformers einops accelerate bitsandbytes


In [3]:
!pip install tokenizers langchain-community
!pip -q install unstructured
!pip install datasets
!pip install chromadb
!pip install sentence-transformers
!pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.9 MB/s eta 0:00:0

In [4]:
# import sys
# import os
import torch
# import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.schema import Document

In [6]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
!pip install datasets
from datasets import load_dataset
# Load the dataset
dataset_name = "teksingh/NagarGPT-dataset0"
dataset = load_dataset(dataset_name)

# Create a list of Document objects for retrieval
documents = []
for row in dataset['train']:
    documents.append(Document(
        page_content=row["सहायक"],  # Use 'सहायक' (answers) as content
        metadata={"question": row["मानव"]}  # Use 'मानव' (questions) as metadata
    ))


README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/46.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/352 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['मानव', 'सहायक'],
        num_rows: 352
    })
})

In [9]:
documents

[Document(metadata={'question': 'जन्मदर्ता गर्नको लागी आवश्यक कागजातहरु के के चाहिन्छ'}, page_content='जन्मदर्ताको लागी आवश्यक कागजपत्रहरु यस प्रकारका छन्| \n१.हजुरले public.donidcr.gov.np प्रयोग गरी अनलाईन फाराम भरेर लिइएको टोकन नम्बर चाहिन्छ,\n२.आफ्नो आमा बाब्रु को नागरिकता माणपत्रको प्रतिलिपि लिनुपर्ने हुन्छ,\n३.यसको साथै आमाबाबुको विवाह प्रमाणपत्रको प्रतिलिपि पनि आवश्यक पर्छ,\n४.अस्पतालमा जन्म भएकाको हकमा अस्पतालबाट जारी जन्म प्रमाणपत्र लिनुपर्ने हुन्छ,\n५.यदि घरमा जन्म भएको भए खोपकार्ड वा अन्य जन्ममिति खुल्ने कागजातहरु आवश्यक पर्छ,\n६.यदी हजुर बसाईँसराईं गरी आएको भए बसाइँसराईं प्रमाणपत्रको प्रतिलिपि पनि लिनुपर्नेहुन्छ,\n७.तर विदेशमा जन्म भएकाको हकमा सम्बन्धित नेपाली नियोगबाट वा कन्सुलर सेवा विभागबाट प्रमाणित कागजपत्र पनि चाहिन्छ,|'),
 Document(metadata={'question': 'जन्मदर्ताको सेवा शुल्क तथा दस्तुर रकम के कति लाग्छ'}, page_content='जन्मदर्ता संबन्धित शुल्कको विवरण तल दिएको छ|,\n१.जन्म भएको ३५ दिन भित्रमा दर्ता गर्दा शुल्क लाग्दैन,\n२.तर ३५ दिनपछि भने रू.५० शुल्क तिर्नुपर्ने हुन्छ

In [26]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)
text = text_splitter.split_documents(documents)

In [27]:
text[3]

Document(metadata={'question': 'विवाहदर्ता गर्नको लागी आवश्यक कागजातहरु के के चाहिन्छ'}, page_content='विवाहदर्ताको लागी आवश्यक कागजपत्रहरु यस प्रकारका छन्|,\n१.हजुरले public.donidcr.gov.np प्रयोग गरी अनलाईन फाराम भरेर लिइएको टोकन नम्बर चाहिन्छ,\n२.थप दुलाहा र दुलहीको नागरिकता प्रमाणपत्रको प्रतिलिपि पनि लिनुपर्ने हुन्छ,\n३.साथै दुवैजनाको पासपोर्ट साईजको २ प्रति फोटो आवश्यक पर्छ,\n४.यदि बसाइँसराईं गरी आएको भए बसाइँसराईं प्रमाणपत्रको प्रतिलिपि पनि चाहिन्छ,\n५.तर विदेशमा जन्म भएकाको हकमा सम्बन्धित नेपाली नियोगबाट वा कन्सुलर सेवा विभागबाट प्रमाणित कागजपत्र पनि चाहिन्छ|,')

In [10]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=200)
text = text_splitter.split_documents(documents)

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings


In [46]:
embedding_model = HuggingFaceEmbeddings(model_name="distilbert-base-multilingual-cased")


In [47]:
embedding_test = embedding_model.embed_query("स्वास्थ्य सम्बन्धी गतिविधिहरू बारे सूचना उपलब्ध गराउन आवश्यक कागजातहरू के के हुन्")
print("Embedding dimension:", len(embedding_test))


Embedding dimension: 768


In [27]:
pip install scikit-learn


In [36]:
pip install faiss-gpu  # for GPU (optional)


In [59]:
from langchain.embeddings import HuggingFaceEmbeddings  # Or another embedding model
from langchain.vectorstores import FAISS
import faiss
import numpy as np



# Initialize embedding model (Example: HuggingFace)
embedding_model = HuggingFaceEmbeddings(model_name="distilbert-base-cased")



In [70]:
text = dataset['train']['सहायक']


In [71]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # Example of embedding model
from langchain.vectorstores import FAISS
import faiss
import numpy as np



# Get embeddings for each document
embeddings = [embedding_model.embed_query(doc) for doc in text]

# Convert embeddings to numpy array (FAISS requires numpy arrays of type float32)
embeddings_np = np.array(embeddings).astype(np.float32)

# Initialize FAISS index for 768-dimensional vectors (assuming embeddings have 768 dimensions)
dimension = embeddings_np.shape[1]  # If embeddings are 768-dimensional, this will be 768
index = faiss.IndexFlatL2(dimension)  # Use L2 distance for similarity search

# Add embeddings to FAISS index
index.add(embeddings_np)

# Persist FAISS index to disk
faiss.write_index(index, "faiss_index.index")

# Load the persisted FAISS index
loaded_index = faiss.read_index("faiss_index.index")

# Now we need to create a retriever class to integrate FAISS with LangChain
class FAISSRetriever:
    def __init__(self, index, documents):
        self.index = index
        self.documents = documents

    def retrieve(self, query_embedding, k=3):
        query_embedding_np = np.array([query_embedding]).astype(np.float32)
        D, I = self.index.search(query_embedding_np, k)  # k is the number of nearest neighbors
        results = [self.documents[i] for i in I[0]]
        return results, D[0]

# Create the retriever
faiss_retriever = FAISSRetriever(loaded_index, text)

# Example query to search the vector store
query = "नेपालको राजधानी के हो?"

# Get the embedding for the query
query_embedding = embedding_model.embed_query(query)

# Retrieve the most similar documents from FAISS
results, distances = faiss_retriever.retrieve(query_embedding, k=3)

# Print the retrieved documents
print("Retrieved Documents:")
for doc in results:
    print(doc)



Retrieved Documents:
मालपोत वा भूमि कर सोहि दिनमा तिर्न सकिन्छ|,
मोही नामसारी सिफारिसको लागि सेवा शुल्क लाग्दैन।
कोठा खोल्ने काम कानुनले तोकेको म्याद भित्र हुन्छ 


In [16]:
# # Persist the database to disk
# vectordb.persist()
# vectordb = None


<ipython-input-16-04c3cef24fef>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [18]:
# vectordb = Chroma(persist_directory='db', embedding_function=embedding_model)


<ipython-input-18-6a260c2c4acd>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory='db', embedding_function=embedding_model)


In [72]:
Document(page_content='नेपाल दक्षिण एशियामा छ।', metadata={'question': 'नेपाल कहाँ छ?'})


Document(metadata={'question': 'नेपाल कहाँ छ?'}, page_content='नेपाल दक्षिण एशियामा छ।')

In [73]:
# Now we need to create a retriever class to integrate FAISS with LangChain
class FAISSRetriever:
    def __init__(self, index, documents):
        self.index = index
        self.documents = documents

    def retrieve(self, query_embedding, k=3):
        query_embedding_np = np.array([query_embedding]).astype(np.float32)
        D, I = self.index.search(query_embedding_np, k)  # k is the number of nearest neighbors
        results = [self.documents[i] for i in I[0]]
        return results, D[0]


In [64]:
faiss_retriever = FAISSRetriever(loaded_index, text)


In [20]:
# retriever = vectordb.as_retriever(search_kwargs={"k": 4}, embedding=embedding_model)


In [74]:
# Test a query for the retrieval system
query = "स्वास्थ्य सम्बन्धी गतिविधिहरू बारे सूचना उपलब्ध गराउन आवश्यक कागजातहरू के के हुन्"
retrieved_docs, distances = retriever.retrieve(query)


In [75]:
# Retrieve the most similar documents from FAISS
results, distances = faiss_retriever.retrieve(query_embedding, k=3)

# Print the retrieved documents
print("Retrieved Documents:")
for doc in results:
    print(doc)

Retrieved Documents:
मालपोत वा भूमि कर सोहि दिनमा तिर्न सकिन्छ|,
मोही नामसारी सिफारिसको लागि सेवा शुल्क लाग्दैन।
कोठा खोल्ने काम कानुनले तोकेको म्याद भित्र हुन्छ 


In [67]:
print("Retrieved Documents: ", retrieved_docs)


Retrieved Documents:  ['स्वास्थ्य सम्बन्धी गतिविधिहरूको बारेमा सूचना उपलव्ध गराउन कागजातहरू के के चाहिन्छ?', 'नेपालको राजधानी काठमाडौं हो।', 'नेपाल दक्षिण एशियामा छ।', 'स्वास्थ्य सम्बन्धी गतिविधिहरूको बारेमा सूचना उपलव्ध गराउन कागजातहरू के के चाहिन्छ?']


In [82]:
query_result = embeddings.embed_query("How are you")
len(query_result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


512

In [83]:

persist_directory = 'db'


# vectordb = Chroma.from_documents (
#                                 documents=text,
#                                  embedding=embeddings,
#                                  persist_directory=persist_directory)

vectordb = Chroma.from_documents(documents=text,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

InvalidDimensionException: Embedding dimension 512 does not match collection dimensionality 1024

In [66]:
# persiste the db to disk
vectordb.persist()
vectordb = None
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [84]:
retriever = vectordb.as_retriever(search_kwargs={"k": 4},embedding=embeddings)

In [54]:
# retrieved_docs = retriever.invoke("संस्थागत विद्यालयको नाम अथवा स्थान तथा स्वामित्व परिवर्तन गर्न आवश्यक कागजातहरु के के चाहिन्छ")
# retrieved_docs = retriever.invoke("कक्षा ११/१२ मा सस्थागत विद्यालयमा छात्रवृत्तिमा अध्ययन  गरेको विषय प्रमाणीकरणको लागि आवश्यक कागजात के के हुन्")
retrieved_docs = retriever.invoke("स्वास्थ्य सम्बन्धी गतिविधिहरू बारे सूचना उपलब्ध गराउन आवश्यक कागजातहरू के के हुन्")
# retrieved_docs = retriever.invoke("स्थायीबसोबास सिफारिस गर्न कति समय लाग्छ?")
retrieved_docs

AttributeError: 'FAISSRetriever' object has no attribute 'invoke'

In [ ]:
from transformers import AutoTokenizer
import transformers

In [ ]:
# model="meta-llama/Llama-2-7b-chat-hf"
model="facebook/mbart-large-50"
# model = "abhinavkulkarni/meta-llama-Llama-2-7b-chat-hf-w4-g128-awq"
# model = "asprenger/meta-llama-Llama-2-7b-chat-hf-gemm-w4-g128-awq"
tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
pipeline_obj=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,  #reduce size
    trust_remote_code=True,
    device_map="auto",
    # max_length=800,
    max_new_tokens=150,
    do_sample=True,
    top_k=0,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
!pip install langchain_huggingface
from langchain_huggingface import HuggingFacePipeline

In [ ]:
llm=HuggingFacePipeline(pipeline=pipeline_obj, model_kwargs={'temperature':0.4})

In [ ]:
llm.invoke("स्थायीबसोबास सिफारिस गर्न कति समय लाग्छ?")

'स्थायीबसोबास सिफारिस गर्न कति समय लाग्छ? स्थाई तौर पर, स्थाई तौर पर'

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True
                                    )
query = "प्रतिलिपि प्रमाण कागजातहरु बनाउन के के चाहिन्छ"
llm_response = qa_chain(query)
print(llm_response)


{'query': 'प्रतिलिपि प्रमाण कागजातहरु बनाउन के के चाहिन्छ', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nमिलापत्र कागज वा उजुरी दर्ता गर्न रु २०० लाग्छ\n\nअंग्रेजीमा सिफारिस तथा प्रमाणितका लागि रु ५०० लाग्छ\n\nमिलापत्र कागज वा उजुरी दर्ता एक घण्टामा नै हुन्छ\n\nपहिलो पल्ट सम्पत्ति कर तिर्न कानुन अनुसार सेवा शुल्क तथा दस्तुर  लाग्छ\n\nQuestion: प्रतिलिपि प्रमाण कागजातहरु बनाउन के के चाहिन्छ\nHelpful Answer: Komercijojentalo.com/products/viewtopic.php?id=62.html", 'source_documents': [Document(metadata={'question': 'मिलापत्र कागज वा उजुरी दर्ता गर्न लाग्ने सेवा शुल्क कति हो'}, page_content='मिलापत्र कागज वा उजुरी दर्ता गर्न रु २०० लाग्छ'), Document(metadata={'question': 'अंग्रेजीमा सिफारिस तथा प्रमाणितका लागि लाग्ने सेवा शुल्क कति हो'}, page_content='अंग्रेजीमा सिफारिस तथा प्रमाणितका लागि रु ५०० लाग्छ'), Document(metadata={'question': 'मिलापत्र कागज वा उजुरी

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever)

In [ ]:
result=chain({"question": "प्रतिलिपि प्रमाण कागजातहरु बनाउन के के चाहिन्छ"}, return_only_outputs=True)

ValueError: Document prompt requires documents to have metadata variables: ['source']. Received document with missing metadata: ['source'].
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT

In [ ]:
result['answer']